In [1]:
import os
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv
from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#change RAM allocation, current = 8GB
#change Core allocation, current = 3 Cores

spark = SparkSession.builder.appName("Project - Chicago crime")\
.config("spark.some.config.option", "some-value")\
.config("spark.driver.memory", "8g")\
.config("spark.driver.cores", '3')\
.getOrCreate()


In [3]:
crimes_schema = StructType([StructField("ID", StringType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", StringType(), True),
                            StructField("District", StringType(), True),
                            StructField("Ward", StringType(), True),
                            StructField("Community Area", StringType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", DoubleType(), True),
                            StructField("Y Coordinate", DoubleType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", DateType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [4]:
#set your local dataset path
dataSetPath = '/Applications/devActivities/BigDataSpark/Project/Crimes_-_2001_to_present.csv'

In [5]:
dataset = spark.read.option("header", "True")\
        .option("schema", crimes_schema)\
        .csv(dataSetPath)

### Have a look at the data schema:

In [6]:
dataset.limit(20).toPandas().head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11034701,JA366925,01/01/2001 11:00:00 AM,016XX E 86TH PL,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,false,false,...,8,45,11,None,None,2001,08/05/2017 03:50:08 PM,None,None,None
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,false,false,...,21,73,02,None,None,2017,02/11/2018 03:57:41 PM,None,None,None
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,false,false,...,18,70,05,None,None,2017,02/11/2018 03:57:41 PM,None,None,None
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,false,false,...,20,42,06,None,None,2017,02/11/2018 03:57:41 PM,None,None,None
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,false,false,...,42,32,02,None,None,2017,02/11/2018 03:57:41 PM,None,None,None


### Count the total number of instance before preprocessing:

In [7]:
dataset.count()

7092612

### Drop the unused columns:

In [8]:
dataset = dataset.drop('ID')\
            .drop('Case Number')\
            .drop('FBI Code')\
            .drop('Updated On')\
            .drop('IUCR')\
            .drop('X Coordinate')\
            .drop('Y Coordinate')\
            .drop('Location')

### Drop all the instance that have null value, then count:

In [9]:
# Dropping some NA values for now
dataset = dataset.na.drop()
dataset.count()

6413586

### Drop some crime types that are rare and non-criminal:

In [10]:
#Removing these from primary type

dataset = dataset.filter((dataset['Primary Type'] != 'NON-CRIMINAL (SUBJECT SPECIFIED)') &
                         (dataset['Primary Type'] != 'NON-CRIMINAL') &
                         (dataset['Primary Type'] != 'NON - CRIMINAL') &
                         (dataset['Primary Type'] != 'CONCEALED CARRY LICENSE VIOLATION') &
                         (dataset['Primary Type'] != 'DOMESTIC VIOLENCE') &
                         (dataset['Primary Type'] != 'PUBLIC INDECENCY') &
                         (dataset['Primary Type'] != 'OBSCENITY')&
                         (dataset['Primary Type'] != 'RITUALISM'))#Add this type, it is rare. --Haifeng
dataset.count()

6412032

### Define a function that we are going to use to combine similar type of crimes:

In [11]:
def combinePrimaryTypes(targetDf):
    '''
    This function helps combine PRIMARY CRIME TYPES
    '''
    targetDf = targetDf.withColumn("Primary Type", \
              when(targetDf["Primary Type"] == 'OTHER NARCOTIC VIOLATION','NARCOTICS').otherwise(targetDf['Primary Type']))
    targetDf = targetDf.withColumn("Primary Type", \
              when((targetDf["Primary Type"] == 'PROSTITUTION') | (targetDf["Primary Type"] == 'CRIM SEXUAL ASSAULT'),'SEX OFFENSE').otherwise(targetDf['Primary Type']))    
    targetDf = targetDf.withColumn("Primary Type", \
              when((targetDf["Primary Type"] == 'LIQUOR LAW VIOLATION') | (targetDf["Primary Type"] == 'RITUALISM') | (targetDf["Primary Type"] == 'GAMBLING'),'OTHER OFFENSE').otherwise(targetDf['Primary Type']))
    targetDf = targetDf.withColumn("Primary Type", \
              when((targetDf["Primary Type"] == 'CRIMINAL TRESPASS') | (targetDf["Primary Type"] == 'ROBBERY'),'ROBBARY or TRESPASS').otherwise(targetDf['Primary Type']))
    targetDf = targetDf.withColumn("Primary Type", \
              when((targetDf["Primary Type"] == 'INTERFERENCE WITH PUBLIC OFFICER') ,'PUBLIC PEACE VIOLATION').otherwise(targetDf['Primary Type']))
    targetDf = targetDf.withColumn("Primary Type", \
              when((targetDf["Primary Type"] == 'INTIMIDATION') | (targetDf["Primary Type"] == 'STALKING'),'INTIMIDATION or STALKING').otherwise(targetDf['Primary Type']))
    return targetDf

### Apply above function on dataset:

In [12]:
dataset = combinePrimaryTypes(dataset)

### Count the number of types:

In [13]:
typeDF = dataset.groupBy('Primary Type')
count = typeDF.count().count()
print('Count of Primary Type reduced to = ', count)

Count of Primary Type reduced to =  20


### Count the number of every types of crimes:

In [14]:
typeDF.count().orderBy('count', ascending=False).show(count,truncate = False)

+--------------------------+-------+
|Primary Type              |count  |
+--------------------------+-------+
|THEFT                     |1361846|
|BATTERY                   |1174203|
|CRIMINAL DAMAGE           |731523 |
|NARCOTICS                 |654276 |
|ROBBARY or TRESPASS       |426411 |
|OTHER OFFENSE             |424124 |
|ASSAULT                   |404135 |
|BURGLARY                  |367032 |
|MOTOR VEHICLE THEFT       |290723 |
|DECEPTIVE PRACTICE        |255888 |
|SEX OFFENSE               |109315 |
|WEAPONS VIOLATION         |72995  |
|PUBLIC PEACE VIOLATION    |62717  |
|OFFENSE INVOLVING CHILDREN|43664  |
|ARSON                     |10271  |
|HOMICIDE                  |9750   |
|INTIMIDATION or STALKING  |7115   |
|KIDNAPPING                |5693   |
|CRIMINAL SEXUAL ASSAULT   |292    |
|HUMAN TRAFFICKING         |59     |
+--------------------------+-------+

